<img src="pandas_logo.png">

## Agrupación
Al igual que en SQL, pandas permite realizar operaciones de agrupación, las cuales simplifican mucho las cosas.

Imaginemos que tenemos el siguiente dataframe y queremos sumar los valores para los diferentes tipos: 


In [6]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'tipo':['A','B','C','A','B','C','A','B','C'],
                   'valor_1': [0, 5, 10, 5, 10, 15, 10, 15, 20],
                   'valor_2': [10, 3, 10, 4, 10, 1, 2, 15, 6]})

df

,tipo,valor_1,valor_2
0,A,0,10
1,B,5,3
2,C,10,10
3,A,5,4
4,B,10,10
5,C,15,1
6,A,10,2
7,B,15,15
8,C,20,6


Podriamos repetir la siguiente operación para cada una de los tipos:

In [12]:
df.loc[df['tipo'] == 'A', ['valor_1', 'valor_2']].sum()

valor_1    15
valor_2    16
dtype: int64

O mejor, utilizar las funciones de agregación...

In [11]:
df.groupby('tipo').sum()  # esto devuelve un dataframe

,valor_1,valor_2
tipo,,
A,15,16
B,30,28
C,45,17


Si quisieramos diferentes funciones sobre las dos columnas:

In [20]:
df.groupby('tipo').agg({'valor_1': ['min', 'max'], 'valor_2': ['sum','count']})


valor_1     valor_2      
         min max     sum count
tipo                          
A          0  10      16     3
B          5  15      28     3
C         10  20      17     3

## Join - Merge
Los JOINs en SQL sirven para combinar filas de dos o más tablas basándose en un campo común entre ellas. En pandas es lo mismo.

Un ejemplo sencillo te ayudará a entender en que consiste en caso de que no conozcas esta operación:

In [28]:
clientes = pd.DataFrame({'nombre': ['Bob', 'Jake', 'Lisa', 'Sue'],
                        'coche': ['coche_a', 'coche_c', 'coche_b', 'coche_a']}).set_index('coche')
coches = pd.DataFrame({'coche': ['coche_a', 'coche_b', 'coche_c'],
                        'precio': [1000, 2000, 3000]}).set_index('coche')
print(clientes)
print()
print(coches)

        nombre
coche         
coche_a    Bob
coche_c   Jake
coche_b   Lisa
coche_a    Sue

         precio
coche          
coche_a    1000
coche_b    2000
coche_c    3000


Tenemos estas dos tablas, y queremos relacionar el cliente con el precio de su coche, como la variable coche esta presente en las dos tablas, es tan sencillo como:

In [29]:
clientes_coches = clientes.join(coches)

clientes_coches

,nombre,precio
coche,,
coche_a,Bob,1000
coche_a,Sue,1000
coche_b,Lisa,2000
coche_c,Jake,3000


Vemos que las dos tablas se han 'unido' sobre el index de ambos DataFrame. Que pasaría si quisieramos combinar DataFrames que tuvieran en común una columna 
cualquiera y no el index? 

En este caso habría que utilizar **merge()**. De hecho, merge es más versátil ya que permite realizar ambos tipos de operación. Por lo tanto se recomienda el uso de merge:



In [30]:
clientes = pd.DataFrame({'nombre': ['Bob', 'Jake', 'Lisa', 'Sue'],
                        'coche': ['coche_a', 'coche_c', 'coche_b', 'coche_a']})
coches = pd.DataFrame({'coche': ['coche_a', 'coche_b', 'coche_c'],
                        'precio': [1000, 2000, 3000]})
print(clientes)
print()
print(coches)

  nombre    coche
0    Bob  coche_a
1   Jake  coche_c
2   Lisa  coche_b
3    Sue  coche_a

     coche  precio
0  coche_a    1000
1  coche_b    2000
2  coche_c    3000


In [31]:
# merge se encarga de buscar la columna que se tiene en común, aunque se puede especificar
# utilizando el parametro 'on'.
clientes_coches = clientes.merge(coches)

clientes_coches

,nombre,coche,precio
0,Bob,coche_a,1000
1,Sue,coche_a,1000
2,Jake,coche_c,3000
3,Lisa,coche_b,2000


Existe la posibilidad de que queramos combinar tablas para una columna que aunque hace referencia a la mismca cosa en ambas tablas, se llaman de diferente manera.

En este caso, no hace falta que renombremos una de las dos. Basta con utilizar los parametros: **left_on** y **right_on**:

In [32]:
clientes = pd.DataFrame({'nombre': ['Bob', 'Jake', 'Lisa', 'Sue'],
                        'coche': ['coche_a', 'coche_c', 'coche_b', 'coche_a']})
coches = pd.DataFrame({'carro': ['coche_a', 'coche_b', 'coche_c'],
                        'precio': [1000, 2000, 3000]})
print(clientes)
print()
print(coches)

  nombre    coche
0    Bob  coche_a
1   Jake  coche_c
2   Lisa  coche_b
3    Sue  coche_a

     carro  precio
0  coche_a    1000
1  coche_b    2000
2  coche_c    3000


In [35]:
# Se creará una columna redundante que podremos borrar
clientes_coches = clientes.merge(coches, left_on = 'coche', right_on = 'carro')

clientes_coches

,nombre,coche,carro,precio
0,Bob,coche_a,coche_a,1000
1,Sue,coche_a,coche_a,1000
2,Jake,coche_c,coche_c,3000
3,Lisa,coche_b,coche_b,2000


In [36]:
#borramos una de las dos columnas
clientes_coches.drop(columns = ['carro'])

,nombre,coche,precio
0,Bob,coche_a,1000
1,Sue,coche_a,1000
2,Jake,coche_c,3000
3,Lisa,coche_b,2000


### Tipos de join y merge

#### Inner join

El tipo más común de unión, combina dos DataFrames y devuelve uno que contiene solo aquellas filas que tienen valores **coincidentes** entre los dos DataFrames originales.

<img src = 'inner_join.png' style='height:30vh'>

In [38]:
df1 = pd.DataFrame({'nombre': ['Peter', 'Paul', 'Mary'],
                    'comida': ['carne', 'judías', 'pescado']})

df2 = pd.DataFrame({'nombre': ['Mary', 'Joseph'],
                    'bebida': ['vino', 'cerveza']})

print(df1)
print()
print(df2)
                   


  nombre   comida
0  Peter    carne
1   Paul   judías
2   Mary  pescado

   nombre   bebida
0    Mary     vino
1  Joseph  cerveza


In [39]:
# solo contiene la fila con nombre = Mary que aparece en ambas tablas

df1.merge(df2, how = 'inner')

,nombre,comida,bebida
0,Mary,pescado,vino


#### Left / Right join

Si quisieramos conservar por ejemplo toda la información de la tabla izquierda entonces usaríamos el metodo how = 'left', aquellos valores que no esten contenidos en la tabla derecha entonces apareceran como nulos.

En caso de querer conservar toda la información de la tabla derecha sería lo mismo pero al reves: how = 'right'

<img src = 'left_join.png' style='height: 30vh'>

In [40]:
# contiene todas las filas de la tabla de la izquierda (df1)

df1.merge(df2, how = 'left')

,nombre,comida,bebida
0,Peter,carne,NaN
1,Paul,judías,NaN
2,Mary,pescado,vino


#### Outter join

El último caso: en caso de querer conservar todos los datos.
    
<img src = 'outer_join.png' style='height:30vh'>